In [1]:
import pandas as pd
import random
import numpy as np
import copy
import networkx as nx
import matplotlib.pyplot as plt

Questions for Thiru
- If I have an entanglement swapped between (1,2) and (2,3) to create (1,3), can I regenerate the initial edge entanglements. Currently I'm assuming I can.
- I am discarding the oldest entanglements first. I assume this is information we have access too.

In [19]:
class QuantumNetwork:
    """
    Methods:
        countInstancesEndToEndEntanglement() <- counts instances
        reset() 
        increaseAllAges()
        discardOldEntanglements()
        discardEntanglement()
        performEntanglementGeneration()
        attemptEntanglementGeneration() <- pGen on performEntanglementGeneration
        performSwapping()
        attemptSwapping() <- pSwap on performSwapping()
        showMatrix <- Print current.edges
        showGraph <- pdf graph
    """
    
    
    def __init__(self, initialEdges, pGen, pSwap, cutOffAge, maxLinks, users):
        self.initialEdges = copy.deepcopy(initialEdges) 
        self.currentEdges = {} 
        # Edges are always in ascending order (1,3) never (3,1)! 
        # Avoid duplicates! Potential to be a bug
        self.pGen = pGen
        self.pSwap = pSwap
        self.cutOffAge = cutOffAge
        self.maxLinks = maxLinks
        self.users= users
    
    def calcGlobalAverages(self): # To get average rate of entanglement for links
        # To be implemented for our proportional reward function
        pass
    
    def getPossibleActions(self): # Need to n-step!
        pass
    
    def calcReward(self):
        # Temporary reward function that does not consider fairness.
        for edge in self.currentEdges:
            sum += self.countInstancesEndToEndEntanglement(edge) 
        return sum
        
    def countInstancesEndToEndEntanglement(self, desiredEntanglement):
        for edge in self.currentEdges:
            if edge == desiredEntanglement:
                count = len(self.currentEdges[edge])
                print(f"Edge {edge} has count {count}.")
                return count
        print("No end to end entanglement.")
        return 0
    
    def checkAllEndToEndEntanglements(self): # Too implement
        pass
    
    def reset(self):
        self.currentEdges = {}
        print("Quantum network has been reset to the initial state.")

    def increaseAllAges(self):
        print('Increasing age of all entanglements by 1')
        for edge, ages in self.currentEdges.items():
            for i in range(len(ages)):
                ages[i] += 1

    def discardOldEntanglements(self):
        print('Discarding old entanglements based on cutoff age')
        for edge, ages in list(self.currentEdges.items()):
            sorted_ages = sorted(ages, reverse=True)
            for age in sorted_ages:
                if age >= self.cutOffAge:
                    self.discardEntanglement(edge)
                    print(f"Discarding {edge} due to age.")
                else:
                    break

    def discardEntanglement(self, edge):
        if edge in self.currentEdges:
            ages = self.currentEdges[edge]
            if ages:
                highest_age = max(ages)  # Find the oldest entanglement
                ages.remove(highest_age)
                print(f"Discarded entanglement of age {highest_age} from edge {edge}.")
                if not ages:
                    del self.currentEdges[edge]
                    print(f"Edge {edge} removed from currentEdges as it has no entanglements left.")
        else:
            print(f"Attempted to discard non-existent edge {edge}.")


    def performEntanglementGeneration(self, edge):
        self.currentEdges.setdefault(edge, []).append(0) 
        
    def attemptEntanglementGeneration(self):
        print("\nAttempting Entanglement Generation:")
        for edge in self.initialEdges:
            current_links = self.currentEdges.get(edge, [])
            
            if len(current_links) < self.maxLinks and random.random() < self.pGen : # < or <=
                self.performEntanglementGeneration (edge)
                print(edge, self.currentEdges[edge])

    def performSwapping(self, edge1, edge2):
        combined_nodes = list(edge1) + list(edge2)
        unique_nodes = sorted([node for node in combined_nodes if combined_nodes.count(node) == 1])

        if len(unique_nodes) != 2 or edge1 not in self.currentEdges or edge2 not in self.currentEdges:
            print("Invalid edges for swapping: cannot determine new edge.")
            return

        newLink = tuple(unique_nodes)
        old_ages = self.currentEdges.get(edge1, []) + self.currentEdges.get(edge2, [])
        newAge = max(old_ages) if old_ages else 0
        print(f"Creating new edge {newLink} with entanglement age {newAge}.")

        # Add or update the new link with the new entanglement
        if newLink in self.currentEdges:
            if len(self.currentEdges[newLink]) < self.maxLinks:
                self.currentEdges[newLink].append(newAge)
                print(f"Added entanglement to existing edge {newLink}. Updated ages: {self.currentEdges[newLink]}")
            else:
                print(f"Cannot add entanglement to edge {newLink}; maximum links reached.")
        else:
            self.currentEdges[newLink] = [newAge]
            print(f"Created new edge {newLink} with entanglement age {newAge}.")

        self.discardEntanglement(edge1)
        self.discardEntanglement(edge2)

    def attemptSwapping(self, edge1, edge2):
        print(f"\nAttempting Swapping between {edge1} and {edge2}:")
        if edge1 not in self.currentEdges or edge2 not in self.currentEdges:
            print("One or both edges do not exist in currentEdges.")
            return

        if not self.currentEdges[edge1] or not self.currentEdges[edge2]:
            print("One or both edges do not have entanglement to swap.")
            return

        if random.random() <= self.pSwap:
            self.performSwapping(edge1, edge2)
        else:
            print('Swapping not performed based on pSwap probability.')

    def showMatrix(self):
        print("\nCurrent Entanglement Matrix:")
        for edge, ages in self.currentEdges.items():
            print(f"Edge {edge}: Ages {ages}")

    def showGraph(self):
        G = nx.Graph()

        # Add edges with multiple entanglements
        for edge, ages in self.currentEdges.items():
            for age in ages:
                G.add_edge(edge[0], edge[1], age=age)

        pos = nx.spring_layout(G)  # Spring layout for better visualization

        # Draw nodes
        nx.draw_networkx_nodes(G, pos, node_size=700, node_color="skyblue")
        # Draw edges
        nx.draw_networkx_edges(G, pos, width=2, alpha=0.7, edge_color="black")
        # Draw labels for nodes
        nx.draw_networkx_labels(G, pos, font_size=12, font_color="black")
        # Draw edge labels for age
        edge_labels = {(u, v): f"Age: {data['age']}" for u, v, data in G.edges(data=True)}
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)

        # Display the graph
        plt.title("Quantum Network Graph")
        plt.axis("off")
        plt.show()


In [23]:
# Env Parameters
cutOffAge = 5
pSwap = 1
pGen= 0.4
maxLinks = 2 # Multiplexing
users = [(1,4)]

initialEdges = {  #(edgeNode1, EdgeNode2): [Age]
    (1, 3): [],  
    (2, 3): [],  
    (3, 4): [],  
    (4, 5): [], 
    (4, 6): [],  
}

In [21]:
random.seed(27)
network = QuantumNetwork(initialEdges, pGen, pSwap, cutOffAge, maxLinks, users)

network.attemptEntanglementGeneration()
network.showMatrix()
network.increaseAllAges()
network.discardOldEntanglements()
network.showMatrix()
network.attemptEntanglementGeneration()
network.showMatrix()
network.performSwapping((2,3), (3, 4))
network.showMatrix()
network.attemptEntanglementGeneration()
network.showMatrix()
print('\n')
network.countInstancesEndToEndEntanglement((2,4))


Attempting Entanglement Generation:
(4, 5) [0]
(4, 6) [0]

Current Entanglement Matrix:
Edge (4, 5): Ages [0]
Edge (4, 6): Ages [0]
Increasing age of all entanglements by 1
Discarding old entanglements based on cutoff age

Current Entanglement Matrix:
Edge (4, 5): Ages [1]
Edge (4, 6): Ages [1]

Attempting Entanglement Generation:
(2, 3) [0]
(3, 4) [0]
(4, 6) [1, 0]

Current Entanglement Matrix:
Edge (4, 5): Ages [1]
Edge (4, 6): Ages [1, 0]
Edge (2, 3): Ages [0]
Edge (3, 4): Ages [0]
Creating new edge (2, 4) with entanglement age 0.
Created new edge (2, 4) with entanglement age 0.
Discarded entanglement of age 0 from edge (2, 3).
Edge (2, 3) removed from currentEdges as it has no entanglements left.
Discarded entanglement of age 0 from edge (3, 4).
Edge (3, 4) removed from currentEdges as it has no entanglements left.

Current Entanglement Matrix:
Edge (4, 5): Ages [1]
Edge (4, 6): Ages [1, 0]
Edge (2, 4): Ages [0]

Attempting Entanglement Generation:
(3, 4) [0]

Current Entanglement

1

In [ ]:
# N-Step SARSA Parameters
numEpisodes = 1000
nStep = 2
gamma = 0.99
epsilon = 0.1


# Let Env be class, we can make the policy a class later on
def nSTEP(network, numEpisodes=100):
    for episode in range(numEpisodes):
        actionsTaken = [] # Keep tracks of all the actions taken
        network.reset() # Set the initial state
        initialActions = network.getPossibleActions() # Get initial actions
        action = 
            #  [] w/
                # ((1,2), (2,3)) # Can swap
                # (1,2) can generate entanglement
                    # Need to separate function (i.e. entanglement)
                        # so that i can KNOW if i can generate an entanglement at a given time
                        # Implement destroy later!
                        #

        # Initialise our environment to begin new episode training
        # Set initial 
    

In [ ]:
network = QuantumNetwork(initialEdges, pGen, pSwap, cutOffAge, maxLinks, users)
nStep(network)